In [ ]:
#Pyspark not needed for this script, jsut testing

#import pyspark
#from pyspark.sql import SparkSession

In [6]:
import sys
import json
import time
import os
import csv

In [7]:
import numpy as np
from scipy.sparse import csr_matrix
from scipy.sparse import csc_matrix
from scipy.sparse import lil_matrix
from scipy.sparse import hstack
from scipy.sparse import vstack
import scipy.sparse

In [8]:
data_location_desktop = "/media/sf_vm_dropbox/data/spotify_data/data"
data_location_laptop = "/media/sf_VMDropBox/spotify_data/data"
data_location = data_location_laptop
save_location = os.sep.join((data_location, 'new_new_formatted_data'))
NUM_PLAYLISTS = 1000000
NUM_ARTISTS = 295860 #<- from stats.txt
NUM_TRACKS = 2262292


isdir = os.path.isdir(data_location)
if isdir == False:
    print(data_location, ' does not exist. Will fail.')

isdir = os.path.isdir(save_location)
if isdir == False:
    print(save_location, ' does not exist. Creating.')
    os.mkdir(save_location)


In [ ]:
#Helpful commands:
#type() tells type
#sys.getsizeof(obj) tell size in bytes of obj

In [9]:
#BELOW IS FOR ARTISTS:


filenames = os.listdir(data_location)
playlist_index = 0
artist_dict = dict()
playlist_dict = dict()
artist_id_array = []
total_artist_count = 0
file_index = 0
for filename in sorted(filenames):
    if filename.startswith("mpd.slice.") and filename.endswith(".json"):
        f = open(os.sep.join((data_location, filename)))
        js = f.read()
        f.close()
        mpd_slice = json.loads(js)
        start_time = time.time()
        print('Filename: ', filename)
        charmat = lil_matrix((NUM_ARTISTS,1000))
        playlist_index = 0
        for playlist in mpd_slice["playlists"]:
            #print('p',playlist_index, sep='', end=', ')
            #Per each track in given playlist:
            #save playlsit name w/index for getting later

            #Each playlist needs to be unique, but some have the same name..., make sure to 
            #augment the name if that's the case:            
            if playlist['name'] in playlist_dict:
                pl_name = playlist['name']
                index = 1
                while pl_name in playlist_dict:
                    pl_name = playlist['name'] + '_'+str(index) 
                    index = index + 1
                playlist_dict[pl_name] = playlist_index+(file_index*1000)
            else:
                playlist_dict[playlist['name']] = playlist_index+(file_index*1000)

            for track in playlist['tracks']:                
                artist_name = track['artist_name']
                #if arist is in charmat increment that location, 
                #else save off artist name to create new addition to 
                #charmat
                if artist_name in artist_dict:
                    #Already identified artist
                    artist_index = artist_dict[artist_name]
                    charmat[artist_index, playlist_index] = charmat[artist_index, playlist_index] + 1
                    
                else:
                    #Brand new artist:
                    #add to dictionary w/index
                    artist_dict[artist_name] = total_artist_count
                    #add to matrix
                    charmat[total_artist_count, playlist_index] = 1                    
                    #increment count
                    total_artist_count = total_artist_count + 1
                    if total_artist_count >= NUM_ARTISTS:
                        print('Total artist count is larger than spotify info. Will cause error')                
                

            #increment playlist id:
            playlist_index = playlist_index + 1
        print('Time elapsed for file', filename, '= ', time.time() - start_time)
        fname = 'charmat_pt_' + str(file_index)
        scipy.sparse.save_npz(os.sep.join((save_location, fname)), charmat.tocsc())
        file_index = file_index + 1
#At the end write the artist file
1/0
filename = os.sep.join((save_location, "artist_index_new.csv"))
fobj = open(filename, "w")
w = csv.writer(fobj)
w.writerow(['artist_name', 'index'])
for key, val in artist_dict.items():
    w.writerow([key, val])
fobj.close()
#Also write playlist:
filename_new = os.sep.join((save_location, "playlistname_index_new.csv"))
fobj= open(filename_new, "w")
w = csv.writer(fobj)
w.writerow(['playlist_name', 'index'])
for key, val in playlist_dict.items():
    w.writerow([key, val])
fobj.close()

Filename:  mpd.slice.0-999.json
Time elapsed for file mpd.slice.0-999.json =  1.2745039463043213
Filename:  mpd.slice.1000-1999.json
Time elapsed for file mpd.slice.1000-1999.json =  0.5644445419311523
Filename:  mpd.slice.10000-10999.json
Time elapsed for file mpd.slice.10000-10999.json =  0.4834294319152832
Filename:  mpd.slice.100000-100999.json
Time elapsed for file mpd.slice.100000-100999.json =  0.4355759620666504
Filename:  mpd.slice.101000-101999.json
Time elapsed for file mpd.slice.101000-101999.json =  0.9986693859100342
Filename:  mpd.slice.102000-102999.json
Time elapsed for file mpd.slice.102000-102999.json =  0.3988494873046875
Filename:  mpd.slice.103000-103999.json
Time elapsed for file mpd.slice.103000-103999.json =  0.4112100601196289
Filename:  mpd.slice.104000-104999.json
Time elapsed for file mpd.slice.104000-104999.json =  0.7235865592956543
Filename:  mpd.slice.105000-105999.json
Time elapsed for file mpd.slice.105000-105999.json =  0.4124596118927002


ZeroDivisionError: division by zero

In [10]:
#BELOW IS FOR TRACKS

test_index = 5
filenames = sorted(os.listdir(data_location))
filenames = filenames[0:test_index+1]
#filenames = os.listdir(data_location)
playlist_index = 0
track_dict = dict()
playlist_dict = dict()
track_id_array = []
total_track_count = 0
file_index = 0
for filename in sorted(filenames):
    if filename.startswith("mpd.slice.") and filename.endswith(".json"):
        f = open(os.sep.join((data_location, filename)))
        js = f.read()
        f.close()
        mpd_slice = json.loads(js)
        start_time = time.time()
        print('Filename: ', filename)
        charmat = lil_matrix((NUM_TRACKS,1000))
        playlist_index = 0
        for playlist in mpd_slice["playlists"]:
            #print('p',playlist_index, sep='', end=', ')
            #Per each track in given playlist:
            #save playlsit name w/index for getting later

            #Each playlist needs to be unique, but some have the same name..., make sure to 
            #augment the name if that's the case:            
            if playlist['name'] in playlist_dict:
                pl_name = playlist['name']
                index = 1
                while pl_name in playlist_dict:
                    pl_name = playlist['name'] + '_'+str(index) 
                    index = index + 1
                playlist_dict[pl_name] = playlist_index+(file_index*1000)
            else:
                playlist_dict[playlist['name']] = playlist_index+(file_index*1000)

            for track in playlist['tracks']:                
                track_name = track['track_name']
                #if track is in charmat increment that location, 
                #else save off artist name to create new addition to 
                #charmat
                if track_name in track_dict:
                    #Already identified artist
                    track_index = track_dict[track_name]
                    charmat[track_index, playlist_index] = charmat[track_index, playlist_index] + 1
                    
                else:
                    #Brand new track:
                    #add to dictionary w/index
                    track_dict[track_name] = total_track_count
                    #add to matrix
                    charmat[total_track_count, playlist_index] = 1                    
                    #increment count
                    total_track_count = total_track_count + 1
                    if total_track_count >= NUM_TRACKS:
                        print('Total track count is larger than spotify info. Will cause error')                
                

            #increment playlist id:
            playlist_index = playlist_index + 1
        print('Time elapsed for file', filename, '= ', time.time() - start_time)
        fname = 'track_charmat_pt_' + str(file_index)
        scipy.sparse.save_npz(os.sep.join((save_location, fname)), charmat.tocsc())
        file_index = file_index + 1
#At the end write the artist file
filename = os.sep.join((save_location, "track_index_new.csv"))
fobj = open(filename, "w")
w = csv.writer(fobj)
w.writerow(['track_name', 'index'])
for key, val in track_dict.items():
    w.writerow([key, val])
fobj.close()
#Also write playlist:
filename_new = os.sep.join((save_location, "tr_playlistname_index_new.csv"))
fobj= open(filename_new, "w")
w = csv.writer(fobj)
w.writerow(['playlist_name', 'index'])
for key, val in playlist_dict.items():
    w.writerow([key, val])
fobj.close()

Filename:  mpd.slice.0-999.json
Time elapsed for file mpd.slice.0-999.json =  2.733189821243286
Filename:  mpd.slice.1000-1999.json
Time elapsed for file mpd.slice.1000-1999.json =  2.7743563652038574
Filename:  mpd.slice.10000-10999.json
Time elapsed for file mpd.slice.10000-10999.json =  2.7399234771728516
Filename:  mpd.slice.100000-100999.json
Time elapsed for file mpd.slice.100000-100999.json =  1.9884929656982422
Filename:  mpd.slice.101000-101999.json
Time elapsed for file mpd.slice.101000-101999.json =  2.806062936782837
